# TensorFlow Lite: Sentimental Analysis using Amazon review

...

## Package import

In [1]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

!pip uninstall -q -y tensorflow google-colab grpcio
!pip install -q tf-nightly
#!pip install -q git+https://github.com/tensorflow/examples

In [42]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import os
import tensorflow as tf
assert tf.__version__.startswith('2')
import sys
sys.path.append('C:/Users/Adrian/Documents/examples')

from tensorflow_examples.lite.model_customization.core.data_util.text_dataloader import TextClassifierDataLoader
from tensorflow_examples.lite.model_customization.core.model_export_format import ModelExportFormat
import tensorflow_examples.lite.model_customization.core.task.text_classifier as text_classifier

from textdata_extension import from_panda
TextClassifierDataLoader.from_panda = from_panda

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


##  Load Dataset

In [3]:
rev_frame = pd.read_csv("Reviews.csv")

In [4]:
# df is the copy we process for sentimental analysis
df=rev_frame.copy()
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


## Data cleaning and preprocessing

Regard only 'Text' and 'Score' for analysis and rename them to 'review' and 'rating'

In [5]:
df=df[['Text','Score']]
df['review']=df['Text']
df['rating']=df['Score']
df.drop(['Text','Score'],axis=1,inplace=True)

In [6]:
print(df.shape)
df.head()

(568454, 2)


,review,rating
0,I have bought several of the Vitality canned d...,5
1,Product arrived labeled as Jumbo Salted Peanut...,1
2,This is a confection that has been around a fe...,4
3,If you are looking for the secret ingredient i...,2
4,Great taffy at a great price. There was a wid...,5


Check for null values:

In [7]:
print(df['rating'].isnull().sum())
df['review'].isnull().sum()  # no null values.

0


0

Remove duplicates:

In [8]:
# remove duplicates/ for every duplicate we will keep only one row of that type. 
df.drop_duplicates(subset=['rating','review'],keep='first',inplace=True) 

In [9]:
# now check the shape. note that shape is reduced which shows that we did has duplicate rows.
size = df.shape
print(size)
df.head()


(393675, 2)


,review,rating
0,I have bought several of the Vitality canned d...,5
1,Product arrived labeled as Jumbo Salted Peanut...,1
2,This is a confection that has been around a fe...,4
3,If you are looking for the secret ingredient i...,2
4,Great taffy at a great price. There was a wid...,5


In [10]:
def mark_sentiment(rating):
  if(rating<=3):
    return 0
  else:
    return 1

In [11]:
df['sentiment']=df['rating'].apply(mark_sentiment)

In [12]:
df.drop(['rating'],axis=1,inplace=True)
df.head()

,review,sentiment
0,I have bought several of the Vitality canned d...,1
1,Product arrived labeled as Jumbo Salted Peanut...,0
2,This is a confection that has been around a fe...,1
3,If you are looking for the secret ingredient i...,0
4,Great taffy at a great price. There was a wid...,1


In [40]:
df.shape

(100000, 3)

## Cleaning

Has not proven to be more successful!

In [32]:
from bs4 import BeautifulSoup 
from nltk.stem import WordNetLemmatizer  # lammatizer from WordNet
import re
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords  #stopwords

# function to clean and pre-process the text.
def clean_reviews(review):  
    
    # 1. Removing html tags
    review_text = BeautifulSoup(review,"lxml").get_text()
    
    # 2. Retaining only alphabets.
    review_text = re.sub("[^a-zA-Z]"," ",review_text)
    
    # 3. Converting to lower case and splitting
    word_tokens= review_text.lower().split()
    
    # 4. Remove stopwords
    le=WordNetLemmatizer()
    stop_words= set(stopwords.words("english"))     
    word_tokens= [le.lemmatize(w) for w in word_tokens if not w in stop_words]
    
    cleaned_review=" ".join(word_tokens)
    return cleaned_review

## Ignore!

Note that pre processing all the reviews is taking way too much time and so we will take only 100K reviews. To balance the class we have taken equal instances of each sentiment.

In [37]:
pos_df=df.loc[df.sentiment==1,:][:50000]
neg_df=df.loc[df.sentiment==0,:][:50000]

In [38]:
#combining
df=pd.concat([pos_df,neg_df],ignore_index=True)
# shuffling rows
df = df.sample(frac=1).reset_index(drop=True)
print(df.shape)  # perfectly fine.
df.head()

,review,sentiment
0,Good price for a good product. Don't even thi...,1
1,"I just recently started making shaved ice, and...",1
2,"Initially this flavor sounded good, but it has...",0
3,When I first ate one I thought Wow... that is ...,1
4,This cereal smells like chocolate when you ope...,1


In [41]:
df['clean_review']=df['review'].apply(clean_reviews)

,review,sentiment,clean_review
0,Good price for a good product. Don't even thi...,1,good price good product even think getting sup...
1,"I just recently started making shaved ice, and...",1,recently started making shaved ice ordered sev...
2,"Initially this flavor sounded good, but it has...",0,initially flavor sounded good something taste ...
3,When I first ate one I thought Wow... that is ...,1,first ate one thought wow lemony sure liked en...
4,This cereal smells like chocolate when you ope...,1,cereal smell like chocolate open box taste cho...


In the end the cleaning process did not help to improve the accuracy of the model

## Model creation

In the following steps the Panda-object is loaded into an object that Tensorflow can process.
We can choose to process the cleaned reviews or the raw reviews:

In [44]:
train_data, test_data = TextClassifierDataLoader.from_panda(df,pd_label=['clean_review','sentiment']).split(0.6) # The fraction describes the size of the training data

Let's have a look at the data with it's classification and it's review:

In [45]:
for text, label in train_data.dataset.take(10):
  print ("%s: %s"%(train_data.index_to_label[label.numpy()], text.numpy()))

1: b'cat love neighbour cat love aware feeding fish product exclusively unhealthy cat think aside pack also get chicken etc pack balanced diet'
1: b'virtually best cinnamon oil find use sparingly recipe call watered many others like item vendor quality superior service'
0: b'caviar arrives via priority mail usps ice vacuum sealed set four glass jar immediately put fridge later date today caviar small variety despite arriving chilled maintain mild crunch taste salted glance ingredient explains mg sodium x serving one tiny jar oz mg sodium total poor list ingredient stop stabilizer added preservative added three food coloring added may well go grocery store buy shelf rather sent buying please see review fine salmon roe arrives shipped overnight free ice pack company also offer delightful whitefish caviar ship manner reasonable'
1: b'horrendous commute often hour way finding energy level dropping drive needed something enough caffeine give kick since like coffee regular cola job started l

In [57]:
model = text_classifier.create(train_data, epochs=3,num_words=10000)

INFO:tensorflow:Retraining the models...
Train for 1500 steps, validate for 187 steps
Epoch 1/3
1500/1500 [==============================] - 38s 25ms/step - loss: 0.5400 - accuracy: 0.7400 - val_loss: 0.4009 - val_accuracy: 0.831404  - E
Epoch 2/3
1500/1500 [==============================] - 39s 26ms/step - loss: 0.3758 - accuracy: 0.8459 - val_loss: 0.3685 - val_accuracy: 0.8464 - ETA: 29s - loss: 0
Epoch 3/3
1500/1500 [==============================] - 38s 25ms/step - loss: 0.3404 - accuracy: 0.8618 - val_loss: 0.3633 - val_accuracy: 0.84467s - loss: - ETA: 15


While trying out different parameters we found out that the model gives good results with the standard parameters. Increasing the word vocabulary or wordvector dimension lead to overfitting (training loss much smaller than validation loss).

We can have a look at the specifics of the model:

In [52]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 256, 16)           160048    
_________________________________________________________________
global_average_pooling1d_3 ( (None, 16)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 16)                272       
_________________________________________________________________
dropout_3 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 34        
Total params: 160,354
Trainable params: 160,354
Non-trainable params: 0
_________________________________________________________________


After training we should test our model on some unseen test data to evaluate our model:

In [58]:
loss, acc = model.evaluate(test_data)

1250/1250 [==============================] - 26s 21ms/step - loss: 0.3613 - accuracy: 0.8490


Finally we can export the model to use it in the App using TensorFlow Lite :)

In [59]:
model.export('amazon_review_classifier_clean.tflite', 'text_label_c.txt', 'vocab_c.txt')

INFO:tensorflow:Export to tflite model amazon_review_classifier_clean.tflite, saved labels in text_label_c.txt.
INFO:tensorflow:  Saved vocabulary in vocab_c.txt.


In [60]:
# Read TensorFlow Lite model from TensorFlow Lite file.
with tf.io.gfile.GFile('amazon_review_classifier_clean.tflite', 'rb') as f:
  model_content = f.read()

# Read label names from label file.
with tf.io.gfile.GFile('text_label_c.txt', 'r') as f:
  label_names = f.read().split('\n')

# Initialze TensorFlow Lite inpterpreter.
interpreter = tf.lite.Interpreter(model_content=model_content)
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]['index']
output = interpreter.tensor(interpreter.get_output_details()[0]["index"])

# Run predictions on each test data and calculate accuracy.
accurate_count = 0
for i, (text, label) in enumerate(model.test_data.dataset):
    # Pre-processing should remain the same.
    text, label = model.preprocess(text, label)
    # Add batch dimension and convert to float32 to match with the model's input
    # data format.
    text = tf.expand_dims(text, 0).numpy()
    text = tf.cast(text, tf.float32)

    # Run inference.
    interpreter.set_tensor(input_index, text)
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the label with highest
    # probability.
    predict_label = np.argmax(output()[0])
    # Get label name with label index.
    predict_label_name = label_names[predict_label]
    
    accurate_count += (predict_label == label.numpy())

accuracy = accurate_count * 1.0 / model.test_data.size
print('TensorFlow Lite model accuracy = %.4f' % accuracy)

TensorFlow Lite model accuracy = 0.8388
